In [1]:
import pandas as pd

l_test = []
l_test.append(None)
l_test

[None]

In [2]:
len(l_test)

1

In [4]:
import re

price_str = "£2,400 pcm"
match = re.search(r'\d[\d,]*', price_str)

if match:
    price = int(match.group(0).replace(',', ''))
    print(price)  # Output: 2400
else:
    price = None


2400


In [7]:
import time
import datetime

ts = time.time()
print(ts)
ct = datetime.datetime.now()
print("current time:", ct)

1747491545.5240908
current time: 2025-05-17 15:19:05.524587


In [9]:
import pandas as pd

df = pd.DataFrame({"col1":["a","a","b"],
                   "col2":[1,1,2]})

df.drop_duplicates()

,col1,col2
0,a,1
2,b,2


In [13]:

a =  False
not a

True

In [11]:
len(df)

3

In [10]:
ts = time.time()
ct = datetime.datetime.now()
print("current time:", str(ct).replace(" ","|"))

current time: 2025-05-17|15:52:02.520054
